In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OutputCodeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm

# Attribute information

Each record is an example of a hand consisting of five playing cards drawn from a standard deck of 52. Each card is described using two attributes (suit and rank), for a total of 10 predictive attributes.

* ranks (C1, C2, C3, C4, C5)    
  - 1: Ace  
  - 2-10: 2-10  
  - 11: Jack  
  - 12: Queen  
  - 13: King  
* suits (S1, S2, S3, S4, S5)   
  - 1: Hearts  
  - 2: Spades  
  - 3: Diamonds  
  - 4: Clubs  
* hand  
  - 0: Nothing in hand; not a recognized poker hand 
  - 1: One pair; one pair of equal ranks within five cards
  - 2: Two pairs; two pairs of equal ranks within five cards
  - 3: Three of a kind; three equal ranks within five cards
  - 4: Straight; five cards, sequentially ranked with no gaps
  - 5: Flush; five cards with the same suit
  - 6: Full house; pair + different rank three of a kind
  - 7: Four of a kind; four equal ranks within five cards
  - 8: Straight flush; straight + flush
  - 9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush

# Get the dataset

In [2]:
# dataset location
testing_data_location = "./dataset/poker-hand-testing.data"
training_data_location = "./dataset/poker-hand-training-true.data"

# assign column names to the dataset attributes
names = ['S1','C1','S2','C2','S3','C3','S4','C4','S5','C5','hand']

# get testing & training data
testing = pd.read_csv(testing_data_location, names=names, sep=',', header=None)
training = pd.read_csv(training_data_location, names=names, sep=',', header=None)

print("Testing data:", testing.shape)
print("Training data:", training.shape)

# show the first five columns of the training hand
print("-----")
print("Training Hand")
training.head()

Testing data: (1000000, 11)
Training data: (25010, 11)
-----
Training Hand


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9
4,4,1,4,13,4,12,4,11,4,10,9


In [3]:
# show the first five columns of the testing hand
print("Testing Hand")
testing.head()

Testing Hand


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


# Preprocessing

### separate dataset attributes (X) from Class (Y)

In [4]:
# separate first ten columns (X) from eleven'th column for training set
X_train = training.iloc[:,0:10]
Y_train = training.iloc[:,10]

# ...for testing set
X_test = testing.iloc[:,0:10]
Y_test = testing.iloc[:,10]

# show first five columns of training X
print("training X")
X_train.head()

training X


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,10,1,11,1,13,1,12,1,1
1,2,11,2,13,2,10,2,12,2,1
2,3,12,3,11,3,13,3,10,3,1
3,4,10,4,11,4,1,4,13,4,12
4,4,1,4,13,4,12,4,11,4,10


In [5]:
# show first five columns of training Y
print("training Y")
Y_train.head()

training Y


0    9
1    9
2    9
3    9
4    9
Name: hand, dtype: int64

### show unique Class values

In [6]:
# show unique Class categories
Y_train.unique()

array([9, 8, 1, 0, 4, 3, 2, 5, 6, 7])

# Feature Scaling

In [ ]:
scaler = StandardScaler()

# fit only for training data
scaler.fit(X_train)

# transform training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Training and Predictions

In [18]:
# create 3 layers of 10 nodes each + 1000 iterations
mlp = MLPClassifier(solver='adam', hidden_layer_sizes=(15, 15, 15), max_iter=2000, activation='tanh', learning_rate_init=0.02)

# train algorithm on training data
mlp.fit(X_train, Y_train.values.ravel())

# make predictions
predictions = mlp.predict(X_test)

# accuracy score
acc = accuracy_score(Y_test, predictions)

# Check the results

In [19]:
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test, predictions))
print("Accuracy Using MLP: " + str(acc)+'\n')

[[480676  20511      0      0      0     18      0      0      4      0]
 [213452 208265    726     22      0     20      0      1      9      3]
 [  6908  39853    719    142      0      0      0      0      0      0]
 [  5506   3112      1  12398      0      0    102      2      0      0]
 [  2554   1330      0      0      0      1      0      0      0      0]
 [  1898     80      0      0      0     15      0      0      3      0]
 [     0    488     41    868      0      0     27      0      0      0]
 [    16     39      0     90      0      0     48     37      0      0]
 [     8      4      0      0      0      0      0      0      0      0]
 [     3      0      0      0      0      0      0      0      0      0]]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.68      0.96      0.79    501209
           1       0.76      0.49      0.60    422498
           2       0.48      0.02      0.03     47622
           3       0.92      0.59      0.72     21121
           4       0.00      0.00      0.00      3885
           5       0.28      0.01      0.01      1996
           6       0.15      0.02      0.03      1424
           7       0.93      0.16      0.27       230
           8       0.00      0.00      0.00        12
           9       0.00      0.00      0.00         3

   micro avg       0.70      0.70      0.70   1000000
   macro avg       0.42      0.22      0.25   1000000
weighted avg       0.70      0.70      0.67   1000000

Accuracy Using MLP: 0.702137



# Compare with the other models

In [10]:
# define models
#models = [RandomForestClassifier(n_estimators=100), KNeighborsClassifier(), BaggingClassifier(), AdaBoostClassifier(), GaussianNB(), tree.DecisionTreeClassifier(), svm.SVC(kernel='linear', C=1), OutputCodeClassifier(BaggingClassifier()), OneVsRestClassifier(svm.SVC(kernel='linear'))]

# define model names
#model_names = ["Random Forest", "KNeighbors", "Bagging with DT", "AdaBoost", "Naive Bayes", "Decision Tree", "Linear SVM", "OutputCodeClassifier with Linear SVM", "OneVsRestClassifier with Linear SVM"]

# run models
for model, name in zip(models, model_names):
    model.fit(X_train, Y_train.values.ravel())
    
    # make predictions
    predictions = model.predict(X_test)
    
    # show arruracy
    acc = accuracy_score(Y_test, predictions)
    print("Accuracy Using", name,": " + str(acc)+'\n')
    print(classification_report(Y_test, predictions))
    print(confusion_matrix(Y_test, predictions))

NameError: name 'models' is not defined